In [1]:
!pip install pymysql > /dev/null

from google.colab import files
uploaded = files.upload()
filename = list(uploaded.keys())[0]

Saving mysql.json to mysql.json


In [2]:
import json, pymysql
with open(filename) as fp:
    config_str = fp.read()
config = json.loads(config_str)

conn = pymysql.connect(**config)

In [3]:
import random
random.seed(2021)

## 테이블 생성

In [4]:
cur = conn.cursor()
sql_create_table = '''
    CREATE TABLE if NOT EXISTS products(
        pid INT PRIMARY KEY AUTO_INCREMENT,
        pname VARCHAR(32) NOT NULL,
        pprice INT UNSIGNED NOT NULL,
        pcategory VARCHAR(32) NOT NULL,
        pcost INT UNSIGNED NOT NULL
        ) AUTO_INCREMENT=101;
'''

In [5]:
cur.execute(sql_create_table)

0

In [6]:
cur = conn.cursor()
sql_create_table = '''
    CREATE table if NOT EXISTS sales(
        sid INT PRIMARY KEY AUTO_INCREMENT,
        sdate DATE NOT null,
        scompany VARCHAR(32) NOT NULL,
        spid INT NOT NULL,
        sunit INT NOT NULL,
        FOREIGN KEY (spid)
        REFERENCES products(pid) ON UPDATE CASCADE
        );
'''

In [7]:
cur.execute(sql_create_table)

0

## 데이터 추가

- products 테이블에 데이터 추가

In [8]:
categorys = {
    '의류/가방/잡화': ['맨투맨', '니트', '치마', '바지', '지갑', '숄더백', '토트백', '크로스백', '클러치백', '정장', '아우터'],
    '가구' : ['탁자', '선반', '소파', '의자', '식탁', '책상', '옷장', '책장'], 
    '가전제품' : ['TV', '에어컨', '공기청정기', '세탁기', '노트북', '냉장고', '가스레인지', '선풍기', '안마기', '청소기', '제습기', '전기밥솥']
}

In [9]:
sql_insert_ph = "INSERT INTO products(pname, pprice, pcategory, pcost) VALUES (%s, %s, %s, %s);"

In [10]:
for category, pnames in categorys.items():
    for pname in pnames:
        pprice = random.randint(100,999) * 100
        rate = random.randint(85, 98)
        pcost = int(round(pprice * rate / 1000) * 10)
        cur.execute(sql_insert_ph, (pname, pprice, category, pcost))
conn.commit()

- sales 테이블에 데이터 추가

In [11]:
sql_insert_ph = "INSERT INTO sales(sdate, scompany, spid, sunit) VALUES (%s, %s, %s, %s);"

In [12]:
companys = ['쿠팡', '인터파크', '11번가', '티몬', '옥션']

In [13]:
calendar = {1:31,2:29,3:31,4:30,5:31,6:30,7:31,8:31,9:30,10:31,11:30,12:31}

In [14]:
for month, days in calendar.items():
    sales_num = random.randint(10, 20)
    for _ in range(sales_num):
        sdate = f'2020-{month}-{random.randint(1, days)}'
        scompany = random.choice(companys)
        spid = random.randint(101, 131)
        sunit = random.randint(1, 30)
        cur.execute(sql_insert_ph, (sdate, scompany, spid, sunit))
conn.commit()

## 두 테이블을 Join한 View 생성

In [15]:
sql = """
    CREATE VIEW sales_book as
        SELECT sid, sdate, scompany, pid, pname, pcategory, pprice, sunit, pcost,
            pprice*sunit AS revenue, (pprice-pcost)*sunit AS profit
        FROM sales
        JOIN products
        ON sales.spid = products.pid
        ORDER BY sdate;
"""

In [16]:
cur = conn.cursor()
cur.execute(sql)

0

In [17]:
sql = 'select * from sales_book'
cur.execute(sql)
results = cur.fetchall()

In [18]:
import pandas as pd

df = pd.DataFrame(results)
df.head()

,0,1,2,3,4,5,6,7,8,9,10
0,1,2020-01-01,쿠팡,118,옷장,가구,30400,25,28580,760000,45500
1,6,2020-01-03,인터파크,129,청소기,가전제품,65500,19,57640,1244500,149340
2,12,2020-01-04,11번가,101,맨투맨,의류/가방/잡화,95600,15,87000,1434000,129000
3,3,2020-01-06,인터파크,113,선반,가구,78900,14,75740,1104600,44240
4,9,2020-01-09,티몬,106,숄더백,의류/가방/잡화,58500,21,54990,1228500,73710


In [19]:
del df[3]
del df[0]

In [20]:
df.columns = ['판매일자','판매처','상품명','카테고리','단가','수량','원가','매출액','이익']
df.head()

,판매일자,판매처,상품명,카테고리,단가,수량,원가,매출액,이익
0,2020-01-01,쿠팡,옷장,가구,30400,25,28580,760000,45500
1,2020-01-03,인터파크,청소기,가전제품,65500,19,57640,1244500,149340
2,2020-01-04,11번가,맨투맨,의류/가방/잡화,95600,15,87000,1434000,129000
3,2020-01-06,인터파크,선반,가구,78900,14,75740,1104600,44240
4,2020-01-09,티몬,숄더백,의류/가방/잡화,58500,21,54990,1228500,73710


In [21]:
df.to_csv('sales_book.csv', index=False)

## 마무리

In [22]:
cur.close()
conn.close()